# Mecab 사용자사전 추가하기 (PowerShell에 별도실행 대신 파이썬을 이용한 일괄 실행) 

In [1]:
# 종성여부 판단
from jamo import h2j, j2hcj
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic") 

# 추가할 단어 리스트(config 파일 불러오기)-외부로 공개를 원하지 않을 경우 선택
from config import *
_cfg = config
user_dict = _cfg['USER_DICT']
# 개인 사용이라면 아래에 리스트 입력이 더 간편
# user_dict = []

def get_jongsung_TF(sample_text):
    sample_text_list = list(sample_text)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]
    jongsung_TF = "T"
    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
        jongsung_TF = "F"
    return jongsung_TF

#기본 사용자사전 형식(기본값)
with open("C:/mecab/user-dic/nnp.csv", 'r', encoding='utf-8') as f:
    file_data = f.readlines()
file_data

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n', '구글,,,,NNP,*,T,구글,*,*,*,*,*\n']

In [2]:
# 종성여부 값 입력
for word in user_dict:
    jongsung_TF = get_jongsung_TF(word)
    line = '{},,,,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)
    file_data.append(line)
    
# 사용자사전에 쓰기
with open("C:/mecab/user-dic/nnp_new.csv", 'w', encoding='utf-8') as f:
    for line in list(set(file_data)): # 중복제거
        f.write(line)

# powershell이 실행 안 되므로 변수 reset
%reset -f

In [3]:
#사용자사전에 작성한 내용 적용

#reset 했으므로 다시 import
import subprocess

p = subprocess.Popen(
  [
    "powershell.exe", 
    "-noprofile", "-c",
    r"""
    Start-Process -Verb RunAs -Wait powershell.exe -Args "
     -noprofile -c Set-ExecutionPolicy Unrestricted; cd c:\mecab; .\tools\add-userdic-win.ps1;
      "
    """
  ]
)
p.communicate()

(None, None)

In [4]:
# 우선순위 조정

#reset 했으므로 다시 import
import pandas as pd
df_ = pd.read_csv('C:/mecab/mecab-ko-dic/user-nnp_new.csv', header=None)
df_[3] = list(map(lambda x: 0, df_[3]))
df_.to_csv('C:/mecab/mecab-ko-dic/user-nnp_new.csv', header=None, index=False)

p = subprocess.Popen(
  [
    "powershell.exe", 
    "-noprofile", "-c",
    r"""
    Start-Process -Verb RunAs -Wait powershell.exe -Args "
     -noprofile -c Set-ExecutionPolicy Unrestricted; cd c:\mecab; .\tools\compile-win.ps1;
      "
    """
  ]#,shell=True #관리자권한
)
p.communicate()

(None, None)

In [5]:
# 확인
with open("C:/mecab/user-dic/nnp_new.csv", 'r', encoding='utf-8') as f:
    file_new = f.readlines()
len(file_new)

64